<a href="https://colab.research.google.com/github/don05050505/don05050505/blob/main/mnistClassificationHiddenLayertwo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf


class MnistClassificationTwoHiddenLayerModel(tf.keras.Model):
    def __init__(self, numOf1HiddenNode, numOf2HiddenNode, numOfOutput):

        super(MnistClassificationTwoHiddenLayerModel, self).__init__()

        self.hiddenLayer1 = tf.keras.layers.Dense(numOf1HiddenNode,
                                                  activation='relu',
                                                  kernel_initializer=self.weight_initialization(),
                                                  bias_initializer=self.weight_initialization()
                                                  )
        self.hiddenLayer2 = tf.keras.layers.Dense(numOf2HiddenNode,
                                                  activation='relu',
                                                  kernel_initializer=self.weight_initialization(),
                                                  bias_initializer=self.weight_initialization())

        self.outPut = tf.keras.layers.Dense(numOfOutput,
                                            activation=None,
                                            kernel_initializer=self.weight_initialization(),
                                            bias_initializer=self.weight_initialization()
                                            )
    def weight_initialization(self):
        return tf.keras.initializers.RandomNormal(mean=0., stddev=1., seed=None)

    def call(self, x): #forward
        y1 = self.hiddenLayer1(x)
        y2 = self.hiddenLayer2(y1)
        y3 = self.outPut(y2)
        return y3

@tf.function
def cross_entropy(y3, y):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y3, labels=y))

@tf.function
def backward(model, optimizer, x, y):
    with tf.GradientTape() as gt:
        yPred = model(x)
        loss = cross_entropy(yPred, y)
    gradz = gt.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradz, model.trainable_variables))

@tf.function
def accuracy(yPred, y):
    correction = tf.equal(tf.argmax(yPred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(correction, tf.float32))
    return acc


def train_learning(epochs, numBatch, numOfTrain, trainData, model, optimizer, lossFrequence, XTest, yTest):
    for epoch in range(epochs):
        avgLoss = 0.
        totBatch = int(numOfTrain/numBatch)
        for bx, by in trainData:
            backward(model, optimizer, bx, by)
            yPreds = model(bx)
            lossBatch = cross_entropy(yPreds, by)
            avgLoss += lossBatch/totBatch
        if epoch % lossFrequence == 0:
            print(f"Epoch: {epoch+1} ===> Loss: {avgLoss}")
    yPred = model(XTest)
    acc = accuracy(yPred, yTest)
    print(f"Accuracy: {acc}")


In [ ]:
if __name__ == "__main__":
    (XTrain, yTrain), (XTest, yTest) = tf.keras.datasets.mnist.load_data()
    XTrain, XTest = XTrain.astype('float32'), XTest.astype('float32')
    XTrain, XTest = XTrain.reshape([-1, 784]), XTest.reshape([-1, 784])
    XTrain, XTest = XTrain / 255., XTest / 255.
    yTrain, yTest = tf.one_hot(yTrain, depth=10), tf.one_hot(yTest, depth=10)

    learningRate = 0.001
    numOfEpochs = 30
    numOfBatch = 784
    lossFrequence = 1
    numOfInput = 784
    numOf1HiddenNode = 256
    numOf2HiddenNode = 256

    numOfOutput = 10
    numOfTrain = XTrain.shape[0]

    trainData = tf.data.Dataset.from_tensor_slices((XTrain, yTrain))
    trainData = trainData.shuffle(60000).batch(numOfBatch)

    model = MnistClassificationTwoHiddenLayerModel(numOf1HiddenNode, numOf2HiddenNode, numOfOutput)
    optimizer = tf.optimizers.Adam(learningRate)

    train_learning(numOfEpochs, numOfBatch, numOfTrain, trainData, model, optimizer, lossFrequence, XTest, yTest)